## データの読み込み

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

#出力する行数を設定、省略をなくす
pd.set_option('display.max_rows', 500)

#データをまとめて読み込み、訓練データはtrain
train_files = glob.glob('./data/train/*.csv')
train = pd.DataFrame()
for file in train_files:
    train1 = pd.read_csv(file)
    cat_train = pd.concat([train,train1])
    train = cat_train

train.head()

/var/folders/ks/17djw9_d5r3300_0z8cc33sh0000gn/T/ipykernel_12349/1329853670.py:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train1 = pd.read_csv(file)
/var/folders/ks/17djw9_d5r3300_0z8cc33sh0000gn/T/ipykernel_12349/1329853670.py:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train1 = pd.read_csv(file)
/var/folders/ks/17djw9_d5r3300_0z8cc33sh0000gn/T/ipykernel_12349/1329853670.py:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train1 = pd.read_csv(file)
/var/folders/ks/17djw9_d5r3300_0z8cc33sh0000gn/T/ipykernel_12349/1329853670.py:14: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  train1 = pd.read_csv(file)


,ID,種類,地域,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,...,前面道路：方位,前面道路：種類,前面道路：幅員（ｍ）,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装,取引の事情等,取引価格（総額）_log
0,40020197,中古マンション等,NaN,40133,福岡県,福岡市中央区,谷,桜坂,7,２ＬＤＫ,...,NaN,NaN,NaN,第１種中高層住居専用地域,60.0,150.0,2021年第2四半期,改装済,NaN,7.041393
1,40031380,中古マンション等,NaN,40137,福岡県,福岡市早良区,西新,西新,4,１Ｋ,...,NaN,NaN,NaN,商業地域,80.0,400.0,2020年第2四半期,未改装,NaN,6.602060
2,40030436,中古マンション等,NaN,40137,福岡県,福岡市早良区,小田部,室見,18,４ＬＤＫ,...,NaN,NaN,NaN,第１種中高層住居専用地域,60.0,150.0,2020年第4四半期,改装済,NaN,7.397940
3,40015295,中古マンション等,NaN,40132,福岡県,福岡市博多区,竹丘町,雑餉隈,5,２ＬＤＫ,...,NaN,NaN,NaN,商業地域,80.0,400.0,2020年第1四半期,未改装,NaN,7.278754
4,40147771,中古マンション等,NaN,40133,福岡県,福岡市中央区,荒戸,大濠公園,6,３ＤＫ,...,NaN,NaN,NaN,商業地域,80.0,400.0,2007年第3四半期,未改装,NaN,6.929419


In [10]:
#trainデータのカラムの確認
train.columns

Index(['ID', '種類', '地域', '市区町村コード', '都道府県名', '市区町村名', '地区名', '最寄駅：名称',
       '最寄駅：距離（分）', '間取り', '面積（㎡）', '土地の形状', '間口', '延床面積（㎡）', '建築年', '建物の構造',
       '用途', '今後の利用目的', '前面道路：方位', '前面道路：種類', '前面道路：幅員（ｍ）', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等', '取引価格（総額）_log'],
      dtype='object')

## 欠損値多いカラムの削除

In [11]:
#目的変数
"""
'取引価格（総額）_log'
"""

#説明変数
"""
'ID', '市区町村コード', '都道府県名', '地区名', '最寄駅：名称', '最寄駅：距離（分）', '間取り', '面積（㎡）',
        '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）', '容積率（％）', '改装'
"""

###欠損値の削除###
class Delete_Null_Data():
    """
    欠損率が高いcolumnを削除するclass
    """
    
    def __init__(self,df):
        self.df = df
        
    #nullのカウントとnullの%のDataframeを返すメソッド
    def null_counts(self):
        null_counts_df = self.df
        null_counts_df = pd.DataFrame(null_counts_df.isnull().sum(),columns=['null_counts'])
        null_counts_df['null_par'] = null_counts_df['null_counts']/len(self.df)
        return null_counts_df
    
    #nullの多いcolumnを削除するメソッド引数にnullの%を指定　１００％→１　５０％→0.5
    def delete_null_columns(self,null_par=0.9):
        df = self.null_counts()
        re_df = df[df['null_par']>null_par]
        cols = re_df.index.tolist()
        delete_df = self.df.drop(labels=cols,axis=1)
        
        return delete_df


In [12]:
#インスタンス化して訓練データを更新する
Adj_Data=Delete_Null_Data(train)
train=Adj_Data.delete_null_columns()
train.shape

(722574, 20)

In [13]:
#trainデータの型確認
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 722574 entries, 0 to 37378
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ID            722574 non-null  int64  
 1   種類            722574 non-null  object 
 2   市区町村コード       722574 non-null  int64  
 3   都道府県名         722574 non-null  object 
 4   市区町村名         722574 non-null  object 
 5   地区名           721906 non-null  object 
 6   最寄駅：名称        719843 non-null  object 
 7   最寄駅：距離（分）     699417 non-null  object 
 8   間取り           696682 non-null  object 
 9   面積（㎡）         722574 non-null  object 
 10  建築年           701593 non-null  object 
 11  建物の構造         703422 non-null  object 
 12  用途            651713 non-null  object 
 13  今後の利用目的       356654 non-null  object 
 14  都市計画          702756 non-null  object 
 15  建ぺい率（％）       698413 non-null  float64
 16  容積率（％）        698413 non-null  float64
 17  取引時点          722574 non-null  object 
 18  改装   

## データ前処理

In [14]:
'''
---データの前処理を施すメソッド---
名前：　preprocess
引数：　データフレーム
戻り値：　前処理をしたデータフレーム
'''

def preprocess(df):
    #欠損値補完で使うrandomをインポート
    import random
    
    #前処理を施す前のデータを確認
    print('==========Before preprocessing:==========',df.info())
    
    #種類, 市区町村名は削除------------------------------------<
    df=df.drop(['種類','市区町村名'], axis=1)
    
    #カテゴリ変数化------------------------------------<
    use_columns=["都道府県名", "地区名", "最寄駅：名称", "間取り", "建物の構造", "用途", "今後の利用目的", "都市計画", "改装"]
    for col in use_columns:
        df[col] = df[col].astype("category")
    
    
    #最寄駅：距離（分） object型→int型に変換------------------------------------<
    #時間がわからない部分の補完、欠損値は10で補完
    re={
        '30分?60分':'45', 
        '1H30?2H':'105', 
        '1H?1H30':'75', 
        '2H?':'120'
    }
    df['最寄駅：距離（分）']=df['最寄駅：距離（分）'].replace(re)
    df['最寄駅：距離（分）']=df['最寄駅：距離（分）'].fillna('10')
    df['最寄駅：距離（分）']=df['最寄駅：距離（分）'].astype(int)
    
    
    #面積（㎡） object型→int型(単位なし)に変換------------------------------------<
    #面積の欠損値はなし、'2000㎡以上'は'2000'で補完
    #df['面積（㎡）']=df['面積（㎡）'].replace('2000㎡以上','2000')
    #1000以上も削ってもいい
    df = df[df['面積（㎡）'] != '2000㎡以上']
    df['面積（㎡）']=df['面積（㎡）'].astype(int)
    
    
    #建築年 object型→int型(築年数)に変換------------------------------------<
    #欠損値を補完, nanのままだとうまくいかないので文字列でnullと入れておく
    df['建築年']=df['建築年'].fillna('null')
    
    l_k=[]
    now=2022
    y=df['建築年']
    
    for i in range(0,len(y),1):        
        #年号が令和、令和2年=2018+2=2020年
        if y.iloc[i][:2]=='令和':
            if y.iloc[i][3]=='年':
                year=now-(2018+int(y.iloc[i][2]))
            else:
                year=now-(2018+int(y.iloc[i][2])*10+int(y.iloc[i][3])) 
            l_k.append(year)

        
        #年号が平成、平成2年=1988+2=1991年
        elif y.iloc[i][:2]=='平成':
            if y.iloc[i][3]=='年':
                year=now-(1988+int(y.iloc[i][2]))
            else:
                year=now-(1988+int(y.iloc[i][2])*10+int(y.iloc[i][3])) 
            l_k.append(year)
            

        #年号が昭和、昭和2年=1925+2=1927年
        elif y.iloc[i][:2]=='昭和':
            if y.iloc[i][3]=='年':
                year=now-(1925+int(y.iloc[i][2]))
            else:
                year=now-(1925+int(y.iloc[i][2])*10+int(y.iloc[i][3])) 
            l_k.append(year)

        #年号が戦前、1868年から1945年の値をランダムに格納
        #1945年に終戦
        elif y.iloc[i][:2]=='戦前':
            year=now-random.randint(1868, 1945)
            l_k.append(year)

        #年号が欠損、1868年から2021年の値をランダムに格納
        else:
            #year=now-random.randint(1868, 2021)
            year=now-np.median(l_k)
            l_k.append(year)
    #リストを代入       
    df['建築年']=l_k
    
    #取引時点 object型→int型に変換------------------------------------<
    l_t=[]

    y1=df['取引時点']
    
    for i in range(0,len(y1),1):        
        #第1四半期
        if y1.iloc[i][6]=='1':
            torihiki=int(y1.iloc[i][:4])+0.25
            l_t.append(torihiki)

        #第2四半期
        elif y1.iloc[i][6]=='2':
            torihiki=int(y1.iloc[i][:4])+0.50
            l_t.append(torihiki)
            
        #第3四半期
        elif y1.iloc[i][6]=='3':
            torihiki=int(y1.iloc[i][:4])+0.50
            l_t.append(torihiki)
            
        #第4四半期
        else:
            torihiki=int(y1.iloc[i][:4])+0.99
            l_t.append(torihiki)
    #リストを代入        
    df['取引時点']=l_t
    
    #前処理を施した後のデータを確認
    print('==========After preprocessing:==========',df.info())
    
    
    #前処理後のデータを返す
    return df

In [15]:
#前処理した結果を更新して訓練データとして扱う
train=preprocess(train)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 722574 entries, 0 to 37378
Data columns (total 20 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   ID            722574 non-null  int64  
 1   種類            722574 non-null  object 
 2   市区町村コード       722574 non-null  int64  
 3   都道府県名         722574 non-null  object 
 4   市区町村名         722574 non-null  object 
 5   地区名           721906 non-null  object 
 6   最寄駅：名称        719843 non-null  object 
 7   最寄駅：距離（分）     699417 non-null  object 
 8   間取り           696682 non-null  object 
 9   面積（㎡）         722574 non-null  object 
 10  建築年           701593 non-null  object 
 11  建物の構造         703422 non-null  object 
 12  用途            651713 non-null  object 
 13  今後の利用目的       356654 non-null  object 
 14  都市計画          702756 non-null  object 
 15  建ぺい率（％）       698413 non-null  float64
 16  容積率（％）        698413 non-null  float64
 17  取引時点          722574 non-null  object 
 18  改装   

## モデルの実装

In [8]:
#必要なライブラリのインポート
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, mean_absolute_error
from sklearn.model_selection import train_test_split, KFold, cross_validate

import lightgbm as lgb
import optuna
import time

#import optuna.integration.lightgbm as lgb



#訓練データと検証データに分割, 7:3
#train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.3, random_state=1)


In [ ]:
{'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 1.0, 'importance_type': 'split', 'learning_rate': 0.1, 'max_depth': -1, 'min_child_samples': 20, 'min_child_weight': 0.001, 'min_split_gain': 0.0, 'n_estimators': 100, 'n_jobs': -1, 'num_leaves': 31, 'objective': None, 'random_state': None, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'silent': True, 'subsample': 1.0, 'subsample_for_bin': 200000, 'subsample_freq': 0}

In [9]:
#説明変数 ： X
X=train.drop('取引価格（総額）_log', axis=1)
#目的変数 : y、logを消すと幅が大きくなる
y=train['取引価格（総額）_log']
#訓練データと検証データに分割, 7:3
train_val_X, test_X, train_val_y, test_y = train_test_split(X, y, test_size=0.3, random_state=1)
train_X, val_X, train_y, val_y = train_test_split(train_val_X, train_val_y, test_size=0.2, random_state=1)
    

In [16]:
#探索するパラメータの列挙と範囲指定、どのデータにフィットさせるか
def objective(trial):
    #n_estimators=trial.suggest_int('n_estimators', 1000,2000)
    
    dtrain=lgb.Dataset(train_X, label=train_y)
    dval=lgb.Dataset(val_X, label=val_y)
    
    
    param={
        'boosting_type': 'gbdt',
        'objective': 'regression_l1',
        'metrics': 'mae',
        'n_estimators': trial.suggest_int('n_estimators', 4000,5500),
        'num_leaves': trial.suggest_int('num_leaves', 2,8096),
        'max_depth': trial.suggest_int('max_depth', 25,40),
        'learning_rate': trial.suggest_float('learning_rate', 0.001,0.0015),
        'min_child_samples': trial.suggest_int('min_child_samples', 100,150),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.2,0.95),
        'bagging_freq': trial.suggest_int('bagging_freq', 1,5),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.5,0.95),
        'reg_alpha': trial.suggest_float('reg_alpha', 0.01,0.08),
        'reg_lambda': trial.suggest_float('reg_lambda', 1,15),
        'force_row_wise':True  #再現性確保用のパラメータ
    }
    
    gbm=lgb.train(param, dtrain, valid_sets=[dval,dtrain], num_boost_round=50000, early_stopping_rounds=100, verbose_eval=-1)
    """
    result=cross_validate(estimator=gbm,
                                           X=train_X,
                                           y=train_y,
                                           cv=5,
                                           scoring='neg_mean_absolute_error')
    
    val_accuracy=result['test_score'].mean()
    
    
    """
    pred_val=gbm.predict(val_X)
    val_accuracy=mean_absolute_error(pred_val, val_y)
     
    return val_accuracy

In [17]:
s=time.time()
optuna.logging.set_verbosity(optuna.logging.WARNING)    # optunaのlogを非表示にする。

#study = optuna.create_study(direction='maximize')
study = optuna.create_study(direction='minimize')

study.optimize(objective, n_trials=5)
#study.optimize(objective, timeout=10)
e=time.time()

/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Info] Total Bins 11992
[LightGBM] [Info] Number of data points in the train set: 404635, number of used features: 17
[LightGBM] [Info] Start training from score 7.278754


/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 100 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [18]:
print('実行時間: ',e-s)

実行時間:  10692.587603092194


In [19]:
study.best_params, study.best_value

({'n_estimators': 4570,
  'num_leaves': 1392,
  'max_depth': 32,
  'learning_rate': 0.0013061045135083268,
  'min_child_samples': 101,
  'feature_fraction': 0.7578622783610505,
  'bagging_freq': 2,
  'bagging_fraction': 0.7636544447278826,
  'reg_alpha': 0.02532432867048541,
  'reg_lambda': 7.3888535251996315},
 0.07788102349378519)

In [20]:
#ベストモデルで学習
lgb_reg=lgb.LGBMRegressor(
                        n_estimators=study.best_params['n_estimators'],
                        max_depth=study.best_params['max_depth'],
                        num_leaves=study.best_params['num_leaves'],
                        learning_rate=study.best_params['learning_rate'],
                        min_child_samples=study.best_params['min_child_samples'],
                        reg_alpha=study.best_params['reg_alpha'],
                        reg_lambda=study.best_params['reg_lambda'],
                        bagging_freq=study.best_params['bagging_freq'],
                        bagging_fraction=study.best_params['bagging_fraction'],
                        feature_fraction=study.best_params['feature_fraction'],
)
                                           
lgb_reg.fit(train_X, train_y)

pred_train, pred_test=lgb_reg.predict(train_X), lgb_reg.predict(val_X)

[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] feature_fraction is set=0.7578622783610505, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7578622783610505
[LightGBM] [Warning] bagging_fraction is set=0.7636544447278826, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7636544447278826


In [21]:
mean_absolute_error(train_y, pred_train), mean_absolute_error(val_y, pred_test) 

(0.06766728789170447, 0.0778088789998017)

## 自動調整optuna

In [277]:
import optuna.integration.lightgbm as lgb_tune

#trains = lgb.Dataset(train_x, train_y)
#valids = lgb.Dataset(val_x, val_y)

dtrain=lgb.Dataset(train_X, label=train_y)
dval=lgb.Dataset(val_X, label=val_y)

params = {
    "objective": "regression_l1",
    "metric": "mae",
    'n_estimators': 10000, 
    'verbosity':-1,
    'n_jobs':-1
}

s=time.time()
model2 = lgb_tune.train(params, dtrain, 
                    valid_sets=[dtrain, dval],
                    verbose_eval=False,
                    early_stopping_rounds=100,
                    )
e=time.time()
print(e-s)





  0%|                                                     | 0/7 [00:00<?, ?it/s]



feature_fraction, val_score: inf:   0%|                   | 0/7 [00:00<?, ?it/s]/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()'





feature_fraction, val_score: 0.074870:  43%|##1  | 3/7 [44:14<44:12, 663.03s/it]



feature_fraction, val_score: 0.074870:  57%|##8  | 4/7 [44:14<33:32, 670.93s/it]



feature_fraction, val_score: 0.074870:  57%|##8  | 4/7 [44:14<33:32, 670.93s/it]/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is dep





num_leaves, val_score: 0.073779:   0%|                   | 0/20 [41:37<?, ?it/s]



num_leaves, val_score: 0.073779:   5%|3     | 1/20 [41:37<13:10:45, 2497.15s/it]



num_leaves, val_score: 0.073779:   5%|3     | 1/20 [41:37<13:10:45, 2497.15s/it]/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is dep





num_leaves, val_score: 0.073733:  20%|#    | 4/20 [2:56:29<5:27:52, 1229.55s/it]



num_leaves, val_score: 0.073733:  25%|#   | 5/20 [2:56:29<10:54:51, 2619.44s/it]



num_leaves, val_score: 0.073733:  25%|#   | 5/20 [2:56:29<10:54:51, 2619.44s/it]/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is dep





num_leaves, val_score: 0.073691:  40%|#2 | 8/20 [11:45:44<20:06:08, 6030.68s/it]



num_leaves, val_score: 0.073691:  45%|#3 | 9/20 [11:45:44<13:48:55, 4521.42s/it]



num_leaves, val_score: 0.073691:  45%|#3 | 9/20 [11:45:44<13:48:55, 4521.42s/it]/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is dep

...





regularization_factors, val_score: 0.073577:  55%|5| 11/20 [12:25:24<10:36:56, 4



regularization_factors, val_score: 0.073577:  60%|6| 12/20 [12:25:24<9:43:46, 43



regularization_factors, val_score: 0.073577:  60%|6| 12/20 [12:25:24<9:43:46, 43/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is dep





regularization_factors, val_score: 0.073577:  75%|7| 15/20 [16:00:43<5:25:24, 39



regularization_factors, val_score: 0.073577:  80%|8| 16/20 [16:00:43<3:45:34, 33



regularization_factors, val_score: 0.073577:  80%|8| 16/20 [16:00:43<3:45:34, 33/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is dep





regularization_factors, val_score: 0.073577:  95%|9| 19/20 [25:37:48<2:13:05, 79



regularization_factors, val_score: 0.073577: 100%|#| 20/20 [25:37:48<00:00, 4613




  0%|                                                     | 0/5 [00:00<?, ?it/s]



min_data_in_leaf, val_score: 0.073577:   0%|              | 0/5 [00:00<?, ?it/s]/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-0128/opt/anaconda3/lib/python





min_data_in_leaf, val_score: 0.073577:  60%|6| 3/5 [4:07:47<2:08:19, 3849.95s/it



min_data_in_leaf, val_score: 0.073577:  80%|8| 4/5 [4:07:47<1:03:12, 3792.40s/it



min_data_in_leaf, val_score: 0.073577:  80%|8| 4/5 [4:07:48<1:03:12, 3792.40s/it/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is dep

242112.90175700188


In [276]:
best_params1=model.params
best_params1

{'objective': 'regression_l1',
 'metric': 'l1',
 'verbosity': -1,
 'n_jobs': -1,
 'feature_pre_filter': False,
 'lambda_l1': 0.11556940324858182,
 'lambda_l2': 0.019919048716555563,
 'num_leaves': 207,
 'feature_fraction': 0.4,
 'bagging_fraction': 0.77129655283855,
 'bagging_freq': 7,
 'min_child_samples': 20,
 'categorical_column': [2, 3, 4, 6, 9, 10, 11, 12, 16],
 'num_iterations': 5000,
 'early_stopping_round': 100}

In [22]:
params1={
 'objective': 'regression_l1',
 'metric': 'l1',
 'verbosity': -1,
 'n_jobs': -1,
 'feature_pre_filter': False,
 'lambda_l1': 0.11556940324858182,
 'lambda_l2': 0.019919048716555563,
 'num_leaves': 207,
 'feature_fraction': 0.4,
 'bagging_fraction': 0.77129655283855,
 'bagging_freq': 7,
 'min_child_samples': 20,
 'categorical_column': [2, 3, 4, 6, 9, 10, 11, 12, 16],
 'num_iterations': 5000,
 'early_stopping_round': 100}

model1 = lgb.train(params1, trains, valid_sets=valids, num_boost_round=100, early_stopping_rounds=100, verbose_eval=100)
#model2.fit(train_X, train_y)

/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 100 rounds
[100]	valid_0's l1: 0.0804259
[200]	valid_0's l1: 0.077136
[300]	valid_0's l1: 0.0762975
[400]	valid_0's l1: 0.0758597
[500]	valid_0's l1: 0.075568
[600]	valid_0's l1: 0.0753452
[700]	valid_0's l1: 0.0751898
[800]	valid_0's l1: 0.0750453
[900]	valid_0's l1: 0.074919
[1000]	valid_0's l1: 0.0748212
[1100]	valid_0's l1: 0.0747462
[1200]	valid_0's l1: 0.074671
[1300]	valid_0's l1: 0.0746241
[1400]	valid_0's l1: 0.074576
[1500]	valid_0's l1: 0.0745256
[1600]	valid_0's l1: 0.0744928
[1700]	valid_0's l1: 0.074448
[1800]	valid_0's l1: 0.07441
[1900]	valid_0's l1: 0.0743833
[2000]	valid_0's l1: 0.0743521
[2100]	valid_0's l1: 0.0743108
[2200]	valid_0's l1: 0.074281
[2300]	valid_0's l1: 0.074259
[2400]	valid_0's l1: 0.0742298
[2500]	valid_0's l1: 0.0741951
[2600]	valid_0's l1: 0.0741732
[2700]	valid_0's l1: 0.0741543
[2800]	valid_0's l1: 0.0741376
[2900]	valid_0's l1: 0.0741173
[3000]	valid_0's l1: 0.074098
[3100]	valid_0's l1: 0.07407

In [23]:
pred_train, pred_test=model1.predict(train_X), model1.predict(val_X)
mean_absolute_error(train_y, pred_train), mean_absolute_error(val_y, pred_test) 

(0.05036785783578651, 0.07386821538141088)

In [278]:
best_params2=model2.params
best_params2

{'objective': 'regression_l1',
 'metric': 'l1',
 'verbosity': -1,
 'n_jobs': -1,
 'feature_pre_filter': False,
 'lambda_l1': 0.0,
 'lambda_l2': 0.0,
 'num_leaves': 211,
 'feature_fraction': 0.4,
 'bagging_fraction': 0.7872093542436666,
 'bagging_freq': 2,
 'min_child_samples': 20,
 'num_iterations': 10000,
 'early_stopping_round': 100,
 'categorical_column': [2, 3, 4, 6, 9, 10, 11, 12, 16]}

In [16]:
import lightgbm as lgb

#訓練データと検証データに分割, 7:3
#train_X, val_X, train_y, val_y = train_test_split(X, y, test_size=0.3, random_state=1)

trains = lgb.Dataset(train_X, train_y)
valids = lgb.Dataset(val_X, val_y)

params2={
 'objective': 'regression_l1',
 'metric': 'l1',
 'verbosity': -1,
 'n_jobs': -1,
 'feature_pre_filter': False,
 'lambda_l1': 0.0,
 'lambda_l2': 0.0,
 'num_leaves': 211,
 'feature_fraction': 0.4,
 'bagging_fraction': 0.7872093542436666,
 'bagging_freq': 2,
 'min_child_samples': 20,
 'num_iterations': 10000,
 'early_stopping_round': 100,
 'categorical_column': [2, 3, 4, 6, 9, 10, 11, 12, 16]
}

model2 = lgb.train(params2, trains, valid_sets=valids, num_boost_round=100, early_stopping_rounds=100, verbose_eval=10)
#model2.fit(train_X, train_y)

/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-

Training until validation scores don't improve for 100 rounds
[10]	valid_0's l1: 0.16888
[20]	valid_0's l1: 0.120976
[30]	valid_0's l1: 0.102128
[40]	valid_0's l1: 0.0928606
[50]	valid_0's l1: 0.0885852
[60]	valid_0's l1: 0.0854744
[70]	valid_0's l1: 0.083506
[80]	valid_0's l1: 0.0821975
[90]	valid_0's l1: 0.0810947
[100]	valid_0's l1: 0.0803844
[110]	valid_0's l1: 0.079788
[120]	valid_0's l1: 0.0791622
[130]	valid_0's l1: 0.0788008
[140]	valid_0's l1: 0.0784798
[150]	valid_0's l1: 0.0781032
[160]	valid_0's l1: 0.0778421
[170]	valid_0's l1: 0.0776173
[180]	valid_0's l1: 0.0774344
[190]	valid_0's l1: 0.0772705
[200]	valid_0's l1: 0.0771176
[210]	valid_0's l1: 0.0769863
[220]	valid_0's l1: 0.0768537
[230]	valid_0's l1: 0.0767534
[240]	valid_0's l1: 0.0766669
[250]	valid_0's l1: 0.0765851
[260]	valid_0's l1: 0.0765101
[270]	valid_0's l1: 0.0764562
[280]	valid_0's l1: 0.0763998
[290]	valid_0's l1: 0.0763298
[300]	valid_0's l1: 0.0762782
[310]	valid_0's l1: 0.0762263
[320]	valid_0's l1: 0.0

[2680]	valid_0's l1: 0.0740987
[2690]	valid_0's l1: 0.0740959
[2700]	valid_0's l1: 0.0740951
[2710]	valid_0's l1: 0.0740944
[2720]	valid_0's l1: 0.0740938
[2730]	valid_0's l1: 0.0740934
[2740]	valid_0's l1: 0.074091
[2750]	valid_0's l1: 0.0740879
[2760]	valid_0's l1: 0.0740873
[2770]	valid_0's l1: 0.0740849
[2780]	valid_0's l1: 0.074082
[2790]	valid_0's l1: 0.0740786
[2800]	valid_0's l1: 0.074078
[2810]	valid_0's l1: 0.0740763
[2820]	valid_0's l1: 0.0740761
[2830]	valid_0's l1: 0.0740737
[2840]	valid_0's l1: 0.0740731
[2850]	valid_0's l1: 0.0740714
[2860]	valid_0's l1: 0.0740689
[2870]	valid_0's l1: 0.0740673
[2880]	valid_0's l1: 0.0740656
[2890]	valid_0's l1: 0.0740649
[2900]	valid_0's l1: 0.074064
[2910]	valid_0's l1: 0.0740635
[2920]	valid_0's l1: 0.0740619
[2930]	valid_0's l1: 0.07406
[2940]	valid_0's l1: 0.074058
[2950]	valid_0's l1: 0.0740574
[2960]	valid_0's l1: 0.0740532
[2970]	valid_0's l1: 0.0740517
[2980]	valid_0's l1: 0.0740505
[2990]	valid_0's l1: 0.0740494
[3000]	valid_0'

[5340]	valid_0's l1: 0.0738214
[5350]	valid_0's l1: 0.0738209
[5360]	valid_0's l1: 0.0738217
[5370]	valid_0's l1: 0.0738211
[5380]	valid_0's l1: 0.0738201
[5390]	valid_0's l1: 0.0738195
[5400]	valid_0's l1: 0.0738189
[5410]	valid_0's l1: 0.073818
[5420]	valid_0's l1: 0.0738174
[5430]	valid_0's l1: 0.0738178
[5440]	valid_0's l1: 0.073817
[5450]	valid_0's l1: 0.0738168
[5460]	valid_0's l1: 0.0738169
[5470]	valid_0's l1: 0.0738168
[5480]	valid_0's l1: 0.0738158
[5490]	valid_0's l1: 0.0738159
[5500]	valid_0's l1: 0.0738149
[5510]	valid_0's l1: 0.0738136
[5520]	valid_0's l1: 0.073814
[5530]	valid_0's l1: 0.0738138
[5540]	valid_0's l1: 0.0738137
[5550]	valid_0's l1: 0.073812
[5560]	valid_0's l1: 0.0738109
[5570]	valid_0's l1: 0.0738096
[5580]	valid_0's l1: 0.0738085
[5590]	valid_0's l1: 0.0738073
[5600]	valid_0's l1: 0.0738063
[5610]	valid_0's l1: 0.0738057
[5620]	valid_0's l1: 0.0738053
[5630]	valid_0's l1: 0.0738046
[5640]	valid_0's l1: 0.0738033
[5650]	valid_0's l1: 0.073802
[5660]	valid_

In [17]:
pred_train, pred_test=model2.predict(train_X), model2.predict(val_X)
mean_absolute_error(train_y, pred_train), mean_absolute_error(val_y, pred_test) 

(0.04951491863227428, 0.07379083690543371)

In [269]:
model = lgb.train(best_params, dtrain, valid_sets=dval, num_boost_round=5, early_stopping_rounds=100,verbose_eval=-1)

/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/Shota-0128/opt/anaconda3/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[4983]	valid_0's l1: 0.0737413


In [270]:
pred_train, pred_test=model.predict(train_X), model.predict(val_X)
mean_absolute_error(train_y, pred_train), mean_absolute_error(val_y, pred_test) 

(0.05046090251098516, 0.07374127441532068)

In [263]:
best_params

{'objective': 'regression',
 'metric': 'l1',
 'verbosity': -1,
 'n_jobs': -1,
 'feature_pre_filter': False,
 'lambda_l1': 8.681629623453528,
 'lambda_l2': 2.2289120681727828e-05,
 'num_leaves': 74,
 'feature_fraction': 0.42,
 'bagging_fraction': 1.0,
 'bagging_freq': 0,
 'min_child_samples': 10,
 'num_iterations': 1000,
 'early_stopping_round': 100,
 'categorical_column': [2, 3, 4, 6, 9, 10, 11, 12, 16]}

In [53]:
?lightgbm

Object `lightgbm` not found.


## ランダムフォレスト

In [36]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 1.5 MB/s eta 0:00:00a 0:00:01


In [34]:
train_X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 404635 entries, 88223 to 15023
Data columns (total 17 columns):
 #   Column     Non-Null Count   Dtype   
---  ------     --------------   -----   
 0   ID         404635 non-null  int64   
 1   市区町村コード    404635 non-null  int64   
 2   都道府県名      404635 non-null  category
 3   地区名        404236 non-null  category
 4   最寄駅：名称     403083 non-null  category
 5   最寄駅：距離（分）  404635 non-null  int64   
 6   間取り        390023 non-null  category
 7   面積（㎡）      404635 non-null  int64   
 8   建築年        404635 non-null  float64 
 9   建物の構造      393932 non-null  category
 10  用途         364917 non-null  category
 11  今後の利用目的    200206 non-null  category
 12  都市計画       393463 non-null  category
 13  建ぺい率（％）    391028 non-null  float64 
 14  容積率（％）     391028 non-null  float64 
 15  取引時点       404635 non-null  float64 
 16  改装         366775 non-null  category
dtypes: category(9), float64(4), int64(4)
memory usage: 32.5 MB


In [58]:
#ランダムフォレストはカテゴリ変数を扱えないので、一意的な数字に変換
import category_encoders as ce

def original_enc(df):
    list_cols=['都道府県名', '地区名', '最寄駅：名称', '間取り', '建物の構造', '用途', '今後の利用目的', '都市計画', '改装']
    drop_null_cols=['建ぺい率（％）', '容積率（％）']
    #オリジナルエンコーダーのインスタンス生成
    df_cate_encoder=ce.OrdinalEncoder(cols=list_cols, drop_invariant=True)
    
    #一意的な数字に変換したカラムをencoded_listに格納
    encoded_list=df_cate_encoder.fit_transform(df[list_cols])
    
    #元のカラムを削除
    df=df.drop(list_cols, axis=1)
    df=df.drop(drop_null_cols, axis=1)
    #変換したものを結合
    df=pd.concat([df, encoded_list], axis=1)
    
    return df

encoded=original_enc(train_X)
encoded

,ID,市区町村コード,最寄駅：距離（分）,面積（㎡）,建築年,取引時点,都道府県名,地区名,最寄駅：名称,間取り,建物の構造,用途,今後の利用目的,都市計画,改装
88223,14074187,14117,12,65,8.0,2020.25,1,1,1,1,1,1,1,1,1
13499,28085231,28110,15,80,34.0,2015.50,2,2,2,1,1,1,1,2,1
3226,15005647,15103,8,45,33.0,2021.25,3,3,3,2,2,2,1,3,1
14579,11032854,11102,25,60,30.0,2009.50,4,4,4,1,1,1,2,2,2
31403,13083964,13110,13,65,18.0,2016.50,5,5,5,1,1,1,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32533,13051481,13108,12,40,40.0,2016.50,5,9,204,8,3,1,4,6,1
52187,13209768,13118,8,20,14.0,2020.50,5,785,1299,5,1,1,1,6,3
12808,27136446,27128,0,65,20.0,2012.50,11,1385,87,3,3,1,2,3,1
12090,23202990,23222,14,65,22.0,2009.50,17,3141,1887,1,3,1,2,7,2


In [59]:
#カテゴリ変数の変換
train_X_e=original_enc(train_X)
val_X_e=original_enc(val_X)

train_X_e.isnull().sum()

ID           0
市区町村コード      0
最寄駅：距離（分）    0
面積（㎡）        0
建築年          0
取引時点         0
都道府県名        0
地区名          0
最寄駅：名称       0
間取り          0
建物の構造        0
用途           0
今後の利用目的      0
都市計画         0
改装           0
dtype: int64

In [62]:
#ハイパーパラメータの探索
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import time

params={
    'n_estimators': [i for i in range(8,16)],
    'max_depth': [i for i in range(1,5)],
    'max_leaf_nodes': [2**i for i in range(1,5)]
}

gs=GridSearchCV(
    estimator=RandomForestRegressor(),
    param_grid=params,
    error_score='raise',
    cv=5
)

s=time.time()
gs.fit(train_X_e, train_y)
e=time.time()

print('処理時間：', e-s)
print('最適なパラメータ：', gs.best_params_)
print('スコア：', gs.best_score_)

処理時間： 159.74984097480774
最適なパラメータ： {'max_leaf_nodes': 4, 'n_estimators': 10}
スコア： 0.40483832710686113


In [70]:
#ハイパーパラメータのチューニングあり
tuned_params={
    'max_leaf_nodes': 4, 
    'n_estimators': 10
}

rfr_tuned=RandomForestRegressor(**tuned_params)
s=time.time()
rfr_tuned.fit(train_X_e, train_y)
e=time.time()

train_pred_t=rfr_tuned.predict(train_X_e)
valid_pred_t=rfr_tuned.predict(val_X_e)

print('処理時間：', e-s)
print('訓練用：', mean_absolute_error(y_pred=train_pred_t, y_true=train_y))
print('検証用：', mean_absolute_error(y_pred=valid_pred_t, y_true=val_y))

処理時間： 7.203491926193237
訓練用： 0.20547160858690477
検証用： 0.2058977969117137


In [65]:
#パラメータのチューニングなし
rfr_default=RandomForestRegressor()

s=time.time()
rfr_default.fit(train_X_e, train_y)
e=time.time()

#訓練用の予測結果
train_pred=rfr_default.predict(train_X_e)
#検証用の予測結果
valid_pred=rfr_default.predict(val_X_e)

print('処理時間：', e-s)
print('訓練用：', mean_absolute_error(y_pred=train_pred, y_true=train_y))
print('検証用：', mean_absolute_error(y_pred=valid_pred, y_true=val_y))

処理時間： 415.7144100666046
訓練用： 0.03267201150555187
検証用： 0.19285846527557157


## スタッキング 
* model1: lightGBM estimators=5000, 0.0783 
* model2: lightGBM estimators=10000, 0.0779 
* rfr_defalut: RandomForestRegressor デフォルトパラメータ 


* lgb_reg: 上記3つの結果を結合したデータで学習

In [76]:
#model1の予測結果
pred_train_1=model1.predict(train_X)
pred_val_1=model1.predict(val_X)


In [77]:
#model2の予測結果
pred_train_2=model2.predict(train_X)
pred_val_2=model2.predict(val_X)


In [79]:
#rfr_defalutの予測結果
pred_train_3=rfr_default.predict(train_X_e)
pred_val_3=rfr_default.predict(val_X_e)

# #rfr_tunedの予測結果
# pred_train_3=rfr_default.predict(train_X_e)
# pred_val_3=rfr_default.predict(val_X_e)


In [101]:
#説明変数：３つの結果
#目的変数： train_y, val_y

conbine_train_X=pd.DataFrame(
    data=list(zip(pred_train_1, pred_train_2, pred_train_3)),
    columns=['model1','model2','rfr_default']
)
#conbine_train_X

conbine_val_X=pd.DataFrame(
    data=list(zip(pred_val_1, pred_val_2, pred_val_3)),
    columns=['model1','model2','rfr_default']
)
#conbine_val_X

conbined_train_X=conbine_train_X.drop(['rfr_default'], axis=1)
conbined_val_X=conbine_val_X.drop(['rfr_default'], axis=1)
conbined_train_X


,model1,model2
0,7.605665,7.618811
1,7.252416,7.311766
2,6.769531,6.771403
3,7.033178,7.035051
4,7.794024,7.771108
...,...,...
404630,7.202412,7.179579
404631,7.326936,7.322805
404632,7.381297,7.401968
404633,7.230962,7.231027


In [103]:
#lightGBM(conbined_lgb_reg)の構築、実装
import lightgbm as lgb

#ハイパーパラメータ調整なし
conbined_lgb_reg1=lgb.LGBMRegressor()

#ハイパーパラメータ調整あり
best_params={
      'n_estimators': 1654,
      'num_leaves': 94,
      'learning_rate': 0.0012036928663864562,
      'min_child_samples': 147,
      'feature_fraction': 0.9550725428166746,
      'bagging_freq': 4,
      'bagging_fraction': 0.5127621665962854,
      'reg_alpha': 0.01745893295696308,
      'reg_lambda': 0.014072132478405
}
conbined_lgb_reg2=lgb.LGBMRegressor(**best_params)
          
    
#学習
conbined_lgb_reg1.fit(conbined_train_X, train_y)
conbined_lgb_reg2.fit(conbined_train_X, train_y)

#予測
conbined_pred_train1 = conbined_lgb_reg1.predict(conbined_train_X)
conbined_pred_val1 = conbined_lgb_reg1.predict(conbined_val_X)
conbined_pred_train2 = conbined_lgb_reg2.predict(conbined_train_X)
conbined_pred_val2 = conbined_lgb_reg2.predict(conbined_val_X)

print('ハイパーパラメータ調整なし==========')
print('訓練用：', mean_absolute_error(y_pred=conbined_pred_train1, y_true=train_y))
print('検証用：', mean_absolute_error(y_pred=conbined_pred_val1, y_true=val_y))
print('')
print('ハイパーパラメータ調整あり==========')
print('訓練用：', mean_absolute_error(y_pred=conbined_pred_train2, y_true=train_y))
print('検証用：', mean_absolute_error(y_pred=conbined_pred_val2, y_true=val_y))

ハイパーパラメータ調整なし==========
訓練用： 0.05159586631559602
検証用： 0.07471781447524867

ハイパーパラメータ調整あり==========
訓練用： 0.066009641488777
検証用： 0.08259716533118637


## テストデータ

In [18]:
#テストデータ読み込み
test=pd.read_csv('./data/test.csv')
#人口データ読み込み
#population_test=pd.read_csv('population_test.csv')
#提出用サンプルファイル読み込み
sample=pd.read_csv('./data/sample_submission.csv')

#欠損値の多いカラムを除去
#インスタンス化して訓練データを更新する
Adj_Data=Delete_Null_Data(test)
test=Adj_Data.delete_null_columns()

test.head()

,ID,種類,市区町村コード,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,面積（㎡）,建築年,建物の構造,用途,今後の利用目的,都市計画,建ぺい率（％）,容積率（％）,取引時点,改装
0,1000078,中古マンション等,1101,北海道,札幌市中央区,大通西,西１１丁目,1,３ＬＤＫ,75,平成28年,ＲＣ,住宅,住宅,商業地域,80.0,600.0,2021年第3四半期,未改装
1,1000121,中古マンション等,1101,北海道,札幌市中央区,大通西,西１８丁目,0,１ＤＫ,30,昭和52年,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2021年第3四半期,改装済
2,1000123,中古マンション等,1101,北海道,札幌市中央区,大通西,西１８丁目,3,３ＬＤＫ,70,昭和59年,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2021年第3四半期,改装済
3,1000127,中古マンション等,1101,北海道,札幌市中央区,大通西,西１８丁目,2,２ＬＤＫ,50,昭和64年,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2021年第4四半期,未改装
4,1000129,中古マンション等,1101,北海道,札幌市中央区,大通西,西１８丁目,2,１ＤＫ,45,平成3年,ＳＲＣ,住宅,住宅,商業地域,80.0,400.0,2021年第4四半期,改装済


In [19]:
test=preprocess(test)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21005 entries, 0 to 21004
Data columns (total 19 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         21005 non-null  int64  
 1   種類         21005 non-null  object 
 2   市区町村コード    21005 non-null  int64  
 3   都道府県名      21005 non-null  object 
 4   市区町村名      21005 non-null  object 
 5   地区名        21004 non-null  object 
 6   最寄駅：名称     21000 non-null  object 
 7   最寄駅：距離（分）  21000 non-null  object 
 8   間取り        19847 non-null  object 
 9   面積（㎡）      21005 non-null  int64  
 10  建築年        20404 non-null  object 
 11  建物の構造      19776 non-null  object 
 12  用途         15396 non-null  object 
 13  今後の利用目的    20081 non-null  object 
 14  都市計画       20701 non-null  object 
 15  建ぺい率（％）    20596 non-null  float64
 16  容積率（％）     20596 non-null  float64
 17  取引時点       21005 non-null  object 
 18  改装         18032 non-null  object 
dtypes: float64(2), int64(3), object(14)
memory usa

In [20]:
test.columns

Index(['ID', '市区町村コード', '都道府県名', '地区名', '最寄駅：名称', '最寄駅：距離（分）', '間取り', '面積（㎡）',
       '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）', '容積率（％）', '取引時点',
       '改装'],
      dtype='object')

In [104]:
pred_test_1=model1.predict(test)
pred_test_2=model2.predict(test)

# test_e=original_enc(test)
# pred_test_3=rfr_default.predict(test_e)

In [105]:
conbine_test_X=pd.DataFrame(
    data=list(zip(pred_test_1, pred_test_2)),
    columns=['model1','model2']
)

In [106]:
conbined_pred_test=conbined_lgb_reg2.predict(conbine_test_X)

#提出用ファイルに予測結果格納(LightGBM)
sample['取引価格（総額）_log']=conbined_pred_test
sample.to_csv('submission19.csv', index=None)

In [ ]:

conbined_pred_test=conbined_lgb_reg2.predict(conbine_test_X)

#提出用ファイルに予測結果格納(LightGBM)
sample['取引価格（総額）_log']=conbined_pred_test
sample.to_csv('submission20.csv', index=None)

In [21]:
###学習データの予測結果###
pred=model2.predict(test)

#提出用ファイルに予測結果格納(LightGBM)
sample['取引価格（総額）_log']=pred
sample.to_csv('submission15.csv', index=None)


In [24]:
###学習データの予測結果###
pred=model1.predict(test)

#提出用ファイルに予測結果格納(LightGBM)
sample['取引価格（総額）_log']=pred
sample.to_csv('submission16.csv', index=None)


In [25]:
#15,16の平均
df15=pd.read_csv('submission15.csv')
df16=pd.read_csv('submission16.csv')

pred15=df15['取引価格（総額）_log']
pred16=df16['取引価格（総額）_log']

pred1516=(pred15+pred16)/2

#提出用ファイルに予測結果格納(LightGBM)
sample['取引価格（総額）_log']=pred1516
sample.to_csv('submission1516.csv', index=None)

In [26]:
df1516=pd.read_csv('submission1516.csv')
pred1516=df1516['取引価格（総額）_log']

pred17=(pred15+pred16+pred1516)/3

#提出用ファイルに予測結果格納(LightGBM)
sample['取引価格（総額）_log']=pred17
sample.to_csv('submission17.csv', index=None)